In [1]:
import matplotlib.pyplot as plt

In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import argparse
import time, tqdm
import mymodels
import utils
import warnings
import keras
from keras.callbacks import TensorBoard
import logging

slack_message = ""

from datetime import datetime 

start_time = datetime.now() 


# In[2]:


def model_define(args, metadata):
    num_features = metadata['num_features']
    num_neighbors = metadata['num_neighbors']

    X = layers.Input(shape=(num_features,), dtype=tf.float32) # X_feature
    S = layers.Input(shape=(num_neighbors,), dtype=tf.int32) # Neighbor index
    
    tmp_model = mymodels.str_to_class(args.model_name)(args, metadata)
    Y = tmp_model(X, S) * metadata['y_std'] + metadata['y_mean']
    
    model = keras.models.Model((X, S), Y)
    model_name = tmp_model.model_name

    return model, model_name


# In[3]:


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description='parameter')
    parser.add_argument('--dataset', type=str, choices=['fc', 'kc', 'poa', 'sp'], default='kc')
    parser.add_argument('--max_category', type=int, default=30)
    parser.add_argument('--neighbor_threshold', type=float, default=999)
    parser.add_argument('--hide_loc', action='store_true')
    parser.add_argument('--model_name', type=str, default=f'MyBasicNeighbor')
    parser.add_argument('--restore_model', action='store_true')
    parser.add_argument('--train_again', action='store_true')
    parser.add_argument('--D', type=int, default=128) # hidden dimension
    parser.add_argument('--L', type=int, default=3) # stack of layers
    parser.add_argument('--val_ratio', type=float, default=0.05)
    
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--max_epoch', type=int, default=100)
    parser.add_argument('--optimizer', type=str, default=f'adam')
    parser.add_argument('--learning_rate', type=float, default=0.01)
    parser.add_argument('--patience_stop', type=int, default=10)
    parser.add_argument('--patience_lr', type=int, default=3)
    
    args = parser.parse_args([])
    
    if not os.path.isdir('prediction'):
        os.mkdir('prediction')
    if not os.path.isdir(f'prediction/{args.dataset}'):
        os.mkdir(f'prediction/{args.dataset}')
        
        
    dataset, metadata = utils.dataloader.load_data(args)
    X_train, S_train, y_train, X_val, S_val, y_val, X_test, S_test, y_test = dataset
    print('X_train:', X_train.shape,  '\tS_train:', S_train.shape,  '\ty_train:', y_train.shape)
    print('X_val:', X_val.shape,      '\tS_val:', S_val.shape,      '\ty_val:', y_val.shape)
    print('X_test:', X_test.shape,    '\tS_test:', S_test.shape,    '\ty_test:', y_test.shape)
    print('X_feat_categories:', len(metadata['categories']), metadata['categories'])
    
    
    print(args)
    slack_message += str(args) + '\n'
    
    
    model, model_name = model_define(args, metadata)
    model_logging_name = f'{args.dataset}_{model_name}_{args.neighbor_threshold}_{args.D}'
    model_checkpoint = f'./model_checkpoint/{args.dataset}/{model_logging_name}'
    model_logs = f'./model_logs/{args.dataset}/{model_logging_name}'
    
    if args.optimizer == 'sgd':
        optimizer = tf.keras.optimizers.SGD(learning_rate = args.learning_rate)
    elif args.optimizer == 'adam':
        optimizer = keras.optimizers.Adam(args.learning_rate)
    elif args.optimizer == 'adagrad':
        optimizer = keras.optimizers.Adagrad(args.learning_rate)

    model.compile(loss='mae', optimizer=optimizer)
    model.summary()

    # Define some callbacks to improve training.
    early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=args.patience_stop)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=args.patience_lr)
    model_ckpt = tf.keras.callbacks.ModelCheckpoint(model_checkpoint, save_weights_only=True, \
                    save_best_only=True, monitor='val_loss', mode='min', verbose=0)
#     time_callback = utils.TimeHistory()
    # tb_callback = TensorBoard(log_dir=model_logs, histogram_freq=1, write_graph=True, write_images=True)
#     logging_callback = LoggingCallback()


    # Custom callback for logging metrics during training and testing
    class LoggingCallback(keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs=None):
            if (epoch+1) % 5 == 0:
                predY = self.model.predict((X_test, S_test), batch_size=args.batch_size)
                y_pred = model.predict((X_test, S_test), batch_size=args.batch_size)
                print(f'Test epoch: {epoch+1}', args.dataset, utils.metric(np.exp(y_test.ravel()), np.exp(y_pred.ravel())))

    logging_callback = LoggingCallback()


    model.fit((X_train, S_train), y_train,
                batch_size=args.batch_size,
                epochs=args.max_epoch,
                verbose=1,
                validation_data=((X_val, S_val), y_val),
                callbacks=[early_stopping, model_ckpt, reduce_lr, logging_callback],
    )

    model, model_name = model_define(args, metadata)
    model.load_weights(model_checkpoint)
    
    y_pred = model.predict((X_test, S_test), batch_size=args.batch_size)
    print(model_name, args.dataset, utils.metric(np.exp(y_test.ravel()), np.exp(y_pred.ravel())))

    np.save(f'prediction/{args.dataset}/ground_truth.npy', y_test.ravel())
    np.save(f'prediction/{args.dataset}/{model_logging_name}.npy', y_pred.ravel())



2023-08-24 16:38:26.131273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


X_train: (16422, 18) 	S_train: (16422, 60) 	y_train: (16422,)
X_val: (864, 18) 	S_val: (864, 60) 	y_val: (864,)
X_test: (4322, 18) 	S_test: (4322, 60) 	y_test: (4322,)
X_feat_categories: 18 [0, 0, 13, 0, 0, 0, 6, 2, 5, 5, 12, 0, 0, 0, 0, 0, 0, 0]
Namespace(dataset='kc', max_category=30, neighbor_threshold=999, hide_loc=False, model_name='MyBasicNeighbor', restore_model=False, train_again=False, D=128, L=3, val_ratio=0.05, batch_size=32, max_epoch=100, optimizer='adam', learning_rate=0.01, patience_stop=10, patience_lr=3)


2023-08-24 16:38:27.562343: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-24 16:38:27.562645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-24 16:38:27.562891: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-24 16:38:27.563137: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-24 16:38:27.568158: I tensorflow/compiler/xla/stream_executo

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 18)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 60)]         0           []                               
                                                                                                  
 my_basic_neighbor (MyBasicNeig  (None, 1)           673064      ['input_1[0][0]',                
 hbor)                                                            'input_2[0][0]']                
                                                                                                  
 tf.math.multiply (TFOpLambda)  (None, 1)            0           ['my_basic_neighbor[0][0]']  

2023-08-24 16:38:33.613301: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


514/514 [==============================] - 10s 13ms/step - loss: 0.1766 - val_loss: 0.1594 - lr: 0.0100
Epoch 2/100
514/514 [==============================] - 6s 12ms/step - loss: 0.1500 - val_loss: 0.1564 - lr: 0.0100
Epoch 3/100
514/514 [==============================] - 6s 12ms/step - loss: 0.1458 - val_loss: 0.1511 - lr: 0.0100
Epoch 4/100
514/514 [==============================] - 6s 11ms/step - loss: 0.1399 - val_loss: 0.1716 - lr: 0.0100
Epoch 5/100
136/136 [==============================] - 1s 6ms/step
Test epoch: 5 kc (0.13564917, 137087.98, 0.10045805220899891)
514/514 [==============================] - 8s 16ms/step - loss: 0.1376 - val_loss: 0.1455 - lr: 0.0100
Epoch 6/100
514/514 [==============================] - 6s 11ms/step - loss: 0.1379 - val_loss: 0.1495 - lr: 0.0100
Epoch 7/100
514/514 [==============================] - 6s 12ms/step - loss: 0.1342 - val_loss: 0.1429 - lr: 0.0100
Epoch 8/100
514/514 [==============================] - 6s 12ms/step - loss: 0.1320 - val_

In [3]:
# kc (0.11806576, 117016.77, 0.08231213188728412)
# kc (0.11862325, 115411.625, 0.08378587184873948)
# MyBasicNeighbor, kc (0.11939469, 113397.28, 0.08584784074119675)
# MyBasic kc (0.11889007, 112121.15, 0.08449740417979865)
# MyBasicNeighbor kc (0.11960751, 111863.984, 0.08475349877863886)
